# Jupyter Example
## LAMMPS: granregion/in.granregion.mixer
Reference: https://github.com/lammps/lammps/blob/master/examples/granregion/in.granregion.mixer  
Required files: None  
Description: None

In [1]:
from __future__ import print_function
import random, math
from lammps import IPyLammps

In [2]:
L = IPyLammps()

L.variable("name string mixer")
L.thermo_modify("flush yes")
L.variable("seed equal 14314")

# Particle parameters
L.variable("rlo equal 0.3")
L.variable("rhi equal 0.6")
L.variable("dlo equal 2.0*${rlo}")
L.variable("dhi equal 2.0*${rhi}")
L.variable("skin equal ${rhi}")
L.variable("coeffRes equal 0.1")
L.variable("coeffFric equal 0.5")
L.variable("kn equal 10^5")
L.variable("kt equal 0.2*${kn}")
L.variable("gravity equal 1.0")
L.variable("density equal 1.0")
L.variable("min_mass equal ${density}*4.0/3.0*PI*${rlo}*${rlo}*${rlo}")
L.variable("a equal (-2.0*log(${coeffRes})/PI)^2")
L.variable("gamma_n equal sqrt($a*2*${kn}/${min_mass}/(1+0.25*$a))")
L.variable("gamma_t equal ${gamma_n}*0.5")
L.variable("tcol equal PI/sqrt(2*${kn}/${min_mass}-${gamma_n}/4.0)")
L.variable("dt equal ${tcol}*0.02")
L.timestep("${dt}")

LAMMPS output is captured by PyLammps wrapper


In [3]:
L.variable("dumpfreq equal 10000")
L.variable("logfreq equal 10000")
L.newton("on")
L.atom_style("sphere")
L.boundary("p p f")
L.region("boxreg block 0 20 0 20 0 20")
L.create_box("1 boxreg")
L.pair_style("gran/hertz/history ${kn} ${kt} ${gamma_n} ${gamma_t} ${coeffFric} 1")
L.pair_coeff("* *")
L.neighbor("${skin} bin")
L.thermo("${logfreq}")
L.comm_style("brick")
L.comm_modify("mode multi group all vel yes")
L.balance("1.1 shift xyz 20 1.1")
L.fix("bal all balance 10000 1.1 shift xyz 20 1.01")

In [4]:
# Options specific to pouring

L.region("insreg cylinder z 10 10 8 10 18 side in units box")
L.region("cylreg cylinder z 10 10 10 0 20 side in units box")
L.variable("theta equal (step/400000)*2*PI")
L.region("b1 block 2 18 9 11 0 4 side out rotate v_theta 10 10 0 0 0 1 units box")
L.region("b2 block 9 11 2 18 0 3.99999 side out rotate v_theta 10 10 0 0 0 1 units box")
L.region("mixer intersect 3 cylreg b1 b2 side in")
L.fix("grav all gravity ${gravity} vector 0 0 -1")
L.fix("1 all nve/sphere")
L.fix("mixwall all wall/gran/region hertz/history ${kn} ${kt} ${gamma_n} ${gamma_t} ${coeffFric} 1 region mixer")
L.fix("ins all pour 1000 1 42424 region insreg diam range ${dlo} ${dhi} dens ${density} ${density}")

'Particle insertion: 444 every 84653 steps, 1000 by step 169307'

In [5]:
L.dump("1 all custom ${dumpfreq} ${name}_pour.dump id type mass diameter x y z")

#L.dump("2 all image 4000 image.*.jpg type type axes yes 0.8 0.02 view 60 -30 zoom 1 box no 0.0 axes no 0.0 0.0") 
#L.dump_modify("2 pad 6")

L.dump("3 all movie 4000 mov.mp4 type type axes yes 0.8 0.02 view 60 -30 zoom 1 box no 0.0 axes no 0.0 0.0")
L.dump_modify("3 pad 6")

In [6]:
L.thermo_style("custom step cpu atoms ke v_theta")
L.thermo_modify("flush yes lost warn")
L.run("400000")

['Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 0',
 '  Time step     : 4.72515e-05',
 'Per MPI rank memory allocation (min/avg/max) = 5.862 | 5.862 | 5.862 Mbytes',
 'Step CPU Atoms KinEng v_theta ',
 '       0            0        0           -0            0 ',
 '   10000   0.84202909      444           -0   0.15707963 ',
 '   20000    1.6807182      444           -0   0.31415927 ',
 '   30000     2.526768      444           -0    0.4712389 ',
 '   40000     3.369971      444           -0   0.62831853 ',
 '   50000     4.207839      444           -0   0.78539816 ',
 '   60000     5.046242      444           -0    0.9424778 ',
 '   70000    5.8853202      444           -0    1.0995574 ',
 '   80000    6.7554832      444           -0    1.2566371 ',
 '   90000    8.1062832      888           -0    1.4137167 ',
 '  100000    9.8293722      888           -0    1.5707963 ',
 '  110000    11.557825      888           -0     1.727876 ',
 '  120000    13.296818     

In [7]:
L.unfix("ins")
L.run("400000")

['Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 400000',
 '  Time step     : 4.72515e-05',
 'Per MPI rank memory allocation (min/avg/max) = 13.47 | 13.47 | 13.47 Mbytes',
 'Step CPU Atoms KinEng v_theta ',
 '  400000            0     1000   0.69939385    6.2831853 ',
 '  410000    2.4687631     1000   0.75785303    6.4402649 ',
 '  420000     4.904501     1000   0.74191849    6.5973446 ',
 '  430000    7.3174169     1000   0.69040649    6.7544242 ',
 '  440000    9.7233629     1000   0.64347832    6.9115038 ',
 '  450000    12.186701     1000   0.70259691    7.0685835 ',
 '  460000    14.647315     1000   0.70236655    7.2256631 ',
 '  470000    17.082279     1000   0.66210882    7.3827427 ',
 '  480000    19.550697     1000   0.63565217    7.5398224 ',
 '  490000    22.044347     1000   0.67299426     7.696902 ',
 '  500000    24.465429     1000   0.64012704    7.8539816 ',
 '  510000    26.941103     1000   0.64307242    8.0110613 ',
 '  520000    29.371613

In [8]:
L.undump(3)
L.video("mov.mp4")